In [1]:
# Домашняя работа к 2 уроку курса "Методы сбора"


# 1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы)
# с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input
#  или аргументы). Получившийся список должен содержать в себе минимум:
#
#     *Наименование вакансии
#     *Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
#     *Ссылку на саму вакансию
#     *Сайт откуда собрана вакансия
#
# По своему желанию можно добавить еще работодателя и расположение.
# Данная структура должна быть одинаковая для вакансий с обоих сайтов.
# Общий результат можно вывести с помощью dataFrame через pandas.
#
# 2)Задание похожее, но подход к сбору данных здесь другой.
# Необходимо собрать информацию по продуктам питания с сайтов:
# https://rskrf.ru/ratings/produkty-pitaniya/ и
# https://roscontrol.com/category/produkti/
# Получившийся список должен содержать:
#
#     *Наименование продукта
#     *Категорию продукта        (например Бакалея)
#     *Подкатегорию продукта    (апример "Рис круглозерный")
#     *Параметр "безопасность"
#     *Параметр "качество"
#     *Общий балл
#     *Сайт, откуда получена информация
#
# Данная структура должна быть одинаковая для продуктов с обоих сайтов.
# Общий результат можно вывести с помощью dataFrame через pandas.
#
# Можно выполнить по своему желанию одно любое задание или оба при желании и возможности.


from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
from time import sleep
from random import randint
import pandas as pd
import json


def vacblock(html, attr_vaclist, attr_vacblock):
    parsed_html = bs(html, 'html.parser')
    vaclist = parsed_html.find(name='div', attrs=attr_vaclist)
    if vaclist:
        return vaclist.find_all(name='div', attrs=attr_vacblock)
    else:
        return []


PAGES = int(input('Input how many pages to collect data --> '))
PROFESSION = input('Input profession to search --> ')
PROFESSION_SJ = PROFESSION.replace(' ', '%20')
PROFESSION_HH = PROFESSION.replace(' ', '+')
vacs = []

link_sj = 'https://www.superjob.ru/vacancy/search/?keywords=' + PROFESSION_SJ + '&geo%5Bc%5D%5B0%5D=1&page='
link_hh = 'https://hh.ru/search/vacancy?only_with_salary=false&area=1&clusters=true&search_field=name&enable_snippets=true&salary=&text=' + PROFESSION_HH + '&page='
headers = {'User-agent': 'Mozilla/5.0'}

# SJ
print(f'Search {PAGES} pages on Superjob:')
for i in range(PAGES):
    html = requests.get(link_sj + str(i)).text

    # with open('sj.htm', 'w', encoding='utf-8') as f:
    #    f.write(html)

    # with open('sj.htm', 'r', encoding='utf-8') as f:
    #     html = f.read()

    vacblocks = vacblock(html, {'style': "display:block"}, {'class': "_212By _37XTu"})
    print(f'Page {i+1}, {int(len(vacblocks) / 2)} positions on page')
    if len(vacblocks) == 0:
        break
    for block in vacblocks:
        if vacblocks.index(block) % 2 == 1:
            vacs.append({'vacname': block.find(name='div', attrs={'class': "_3mfro CuJz5 PlM3e _2JVkc _3LJqf"}).text,
                         'salary': block.find(name='span', attrs={'class': re.compile(r"salary*")}).text.replace(u'\xa0', ' '),
                         'link': 'https://www.superjob.ru' + block.find(name='a', href=re.compile(r"/vakansii*"))['href'],
                         'source': 'superjob.ru'
                         })
    sleep(2 + randint(0, 2))

# HH
print(f'Search {PAGES} pages on HH:')
for i in range(PAGES):
    html = requests.get(link_hh + str(i), headers=headers).text

    # with open('hh.htm', 'w', encoding='utf-8') as f:
    #     f.write(html)
    # exit(0)

    # with open('hh.htm', 'r', encoding='utf-8') as f:
    #     html = f.read()

    vacblocks = vacblock(html, {'class': "vacancy-serp"},
                         {'data-qa':
                          ["vacancy-serp__vacancy",
                           "vacancy-serp__vacancy vacancy-serp__vacancy_premium"]})
    print(f'Page {i+1}, {int(len(vacblocks))} positions on page')
    if len(vacblocks) == 0:
        break
    for block in vacblocks:
        sal = block.find(name='div', attrs={'data-qa': "vacancy-serp__vacancy-compensation"})
        if sal:
            sal = sal.text.replace(u'\xa0', ' ')
        else:
            sal = 'Не указана'
        vacs.append({'vacname': block.find(name='a').text,
                     'salary': sal,
                     'link': block.find(name='a')['href'],
                     'source': 'hh.ru'
                     })
    sleep(2 + randint(0, 2))




# with open("vacs.json", "w", encoding="utf-8") as f:
#     json.dump(vacs, f)
# exit(0)

# with open('vacs.json', 'r', encoding='utf-8') as f:
#      vacs = json.load(f)


# узнаём курс доллара и евро с API от ЦБР
linkCB = 'http://www.cbr.ru/scripts/XML_daily.asp?'
headers = {'User-agent': 'Mozilla/5.0'}

req = requests.get(linkCB, headers=headers).text

KURSUSD = float(re.search("США</Name><Value>(\d\d,\d\d\d\d)</Value>", req).groups()[0].replace(',', '.'))
KURSEUR = float(re.search("Евро</Name><Value>(\d\d,\d\d\d\d)</Value>", req).groups()[0].replace(',', '.'))


vacssal = []

for vac in vacs:
    if re.search('USD', vac['salary']):
        KURS = KURSUSD
    elif re.search('EUR', vac['salary']):
        KURS = KURSEUR
    else:
        KURS = 1

    if re.search('\d', vac['salary']):
        if re.search('от', vac['salary']):
            salaryfrom = int(''.join(re.findall('\d', vac['salary']))) * KURS
            salaryto = None
        elif re.search('до', vac['salary']):
            salaryfrom = None
            salaryto = int(''.join(re.findall('\d', vac['salary']))) * KURS
        elif re.search('-|—', vac['salary']):
            salaryfork = re.split('-|—', vac['salary'])
            salaryfrom = int(''.join(re.findall('\d', salaryfork[0]))) * KURS
            salaryto = int(''.join(re.findall('\d', salaryfork[1]))) * KURS
        else:
            salaryfrom = int(''.join(re.findall('\d', vac['salary']))) * KURS
            salaryto = int(''.join(re.findall('\d', vac['salary']))) * KURS
    else:
        salaryfrom = None
        salaryto = None

    vacnew = {'link': vac['link'],
              'vacname': vac['vacname'],
              'salaryfrom': salaryfrom,
              'salaryto': salaryto,
              'source': vac['source']
              }
    vacssal.append(vacnew)

print('All positions on both resources:\n')

data = pd.DataFrame(vacssal)
data.to_pickle('vacssal_DF.pkl')


Input how many pages to collect data --> 2
Input profession to search --> Программист
Search 2 pages on Superjob:
Page 1, 20 positions on page
Page 2, 20 positions on page
Search 2 pages on HH:
Page 1, 20 positions on page
Page 2, 20 positions on page
All positions on both resources:



In [2]:
data

,link,salaryfrom,salaryto,source,vacname
0,https://www.superjob.ru/vakansii/programmist-1...,100000.0,170000.0,superjob.ru,Программист 1С
1,https://www.superjob.ru/vakansii/programmist-1...,50000.0,80000.0,superjob.ru,Программист 1С
2,https://www.superjob.ru/vakansii/programmist-1...,NaN,NaN,superjob.ru,Программист 1С
3,https://www.superjob.ru/vakansii/programmist-c...,70000.0,120000.0,superjob.ru,Программист C#
4,https://www.superjob.ru/vakansii/veb-programmi...,NaN,NaN,superjob.ru,Веб-программист
5,https://www.superjob.ru/vakansii/programmist-1...,100000.0,100000.0,superjob.ru,Программист 1С
6,https://www.superjob.ru/vakansii/programmist-1...,100000.0,100000.0,superjob.ru,Программист 1С
7,https://www.superjob.ru/vakansii/web-programmi...,150000.0,200000.0,superjob.ru,Web-программист / Web-разработчик
8,https://www.superjob.ru/vakansii/programmist-1...,180000.0,200000.0,superjob.ru,Программист 1С
9,https://www.superjob.ru/vakansii/programmist-1...,70000.0,NaN,superjob.ru,Программист 1С
